<a href="https://colab.research.google.com/github/ayulockin/LossLandscape/blob/master/SmallCNN_Cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setups, Installations and Imports

In [1]:
!nvidia-smi

Tue Jul 14 13:53:25 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
## This is so that I can save my models.
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
%%capture
!pip install wandb

In [3]:
# SEED = 666

import tensorflow as tf
# tf.random.set_seed(SEED)

from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import resnet50

import tensorflow_addons as tfa

In [4]:
import os
os.environ["TF_DETERMINISTIC_OPS"] = "1"

import numpy as np
# np.random.seed(SEED)

import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

from mpl_toolkits.axes_grid1 import ImageGrid
from PIL import Image

In [5]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

<IPython.core.display.Javascript object>

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Download and Prepare Dataset

#### CIFAR-10

In [6]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 11s 0us/step


In [7]:
CLASS_NAMES = ("airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

In [8]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


# Dataloader

In [12]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128
IMG_SHAPE = 32

trainloader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
testloader = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def preprocess_image(image, label):
  img = tf.cast(image, tf.float32)
  img = img/255.

  return img, label

trainloader = (
    trainloader
    .shuffle(1024)
    .map(preprocess_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

testloader = (
    testloader
    .map(preprocess_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

# Model

In [9]:
def Model():
  inputs = keras.layers.Input(shape=(IMG_SHAPE, IMG_SHAPE, 3))

  x = keras.layers.Conv2D(16, (3,3), padding='same')(inputs)
  x = keras.activations.relu(x)
  x = keras.layers.MaxPooling2D(2, strides=2)(x)

  x = keras.layers.Conv2D(32,(3,3), padding='same')(x)
  x = keras.activations.relu(x)
  x = keras.layers.MaxPooling2D(2, strides=2)(x)

  x = keras.layers.Conv2D(32,(3,3), padding='same')(x)
  x = keras.activations.relu(x)
  x = keras.layers.MaxPooling2D(2, strides=2)(x)

  x = keras.layers.GlobalAveragePooling2D()(x)
  x = keras.layers.Dense(32, activation='relu')(x)
  x = keras.layers.Dropout(0.1)(x)
  
  outputs = keras.layers.Dense(10, activation='softmax')(x)

  return keras.models.Model(inputs=inputs, outputs=outputs)

# Callbacks

In [10]:
# Custom LR schedule as mentioned in the LossLandscape paper
LR_SCHEDULE = [
    # (epoch to start, learning rate) tuples
    (0, 1.6*1e-3),
    (9, (1.6*1e-3)/2),
    (19, (1.6*1e-3)/4),
    (29, (1.6*1e-3)/8),
]

def lr_schedule(epoch):
    if (epoch >= 0) & (epoch < 9):
        return LR_SCHEDULE[0][1]
    elif (epoch >= 9) & (epoch < 19):
        return LR_SCHEDULE[1][1]
    elif (epoch >= 19) & (epoch < 29):
        return LR_SCHEDULE[2][1]
    else:
        return LR_SCHEDULE[3][1]

lr_callback = tf.keras.callbacks.LearningRateScheduler(lambda epoch: lr_schedule(epoch), verbose=True)

# Initialize Model

In [13]:
keras.backend.clear_session()
model = Model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 16)        448       
_________________________________________________________________
tf_op_layer_Relu (TensorFlow [(None, 32, 32, 16)]      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 32)        4640      
_________________________________________________________________
tf_op_layer_Relu_1 (TensorFl [(None, 16, 16, 32)]      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 32)          0     

# Compile

In [14]:
model.compile('adam', 'sparse_categorical_crossentropy', metrics=['acc'])

# Wandb Init

In [15]:
wandb.init(entity='ayush-thakur', project='loss-landscape')

W&B Run: https://app.wandb.ai/ayush-thakur/loss-landscape/runs/113w8hol

# Train

In [16]:
EPOCHS = 40

_ = model.fit(trainloader,
          epochs=EPOCHS,
          validation_data=testloader,
          callbacks=[WandbCallback(),
                     lr_callback])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.0016.
Epoch 1/40
391/391 [==============================] - 6s 15ms/step - loss: 1.9124 - acc: 0.2755 - val_loss: 1.6872 - val_acc: 0.3756 - lr: 0.0016

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0016.
Epoch 2/40
391/391 [==============================] - 6s 14ms/step - loss: 1.6516 - acc: 0.3821 - val_loss: 1.4989 - val_acc: 0.4459 - lr: 0.0016

Epoch 00003: LearningRateScheduler reducing learning rate to 0.0016.
Epoch 3/40
391/391 [==============================] - 6s 14ms/step - loss: 1.5316 - acc: 0.4298 - val_loss: 1.4783 - val_acc: 0.4502 - lr: 0.0016

Epoch 00004: LearningRateScheduler reducing learning rate to 0.0016.
Epoch 4/40
391/391 [==============================] - 5s 14ms/step - loss: 1.4538 - acc: 0.4589 - val_loss: 1.3666 - val_acc: 0.4890 - lr: 0.0016

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0016.
Epoch 5/40
391/391 [==============================] - 5s 14ms/step

# Evaluate

In [17]:
loss, accuracy = model.evaluate(testloader, callbacks=[WandbCallback()])
print("Test Error Rate: ", round((1-accuracy)*100, 2), '%')

79/79 [==============================] - 1s 7ms/step - loss: 0.9870 - acc: 0.6503
Test Error Rate:  34.97 %
